In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, row_number, sum as spark_sum, max as spark_max, create_map, lit, from_json, explode
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType
from pyspark.sql import Window
from itertools import chain
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
import plotly.graph_objs as go
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_ind

In [2]:
spark = SparkSession.builder.getOrCreate()

# Parse disease list with drug targets

In [ ]:
# Drug targets list prepared based on Pharmaprojects dataset
drug_targets = spark.read.parquet("gs://ot-team/jroldan/analysis/goldStandard_20241031/goldStandardDrugs_outer.parquet/")

In [4]:
# 1. Count unique approvedSymbol values per diseaseId (overall)
all_sum_df = drug_targets.groupBy("diseaseId").agg(F.countDistinct("approvedSymbol").alias("all_sum"))

# 2. Count unique approvedSymbol values per diseaseId and combined_max_phase
pivot_df_chembl = drug_targets.groupBy("diseaseId").pivot("phasesApproved").agg(F.countDistinct("approvedSymbol")).withColumnRenamed("null", "nullChEMBL")
pivot_df_nelson = drug_targets.groupBy("diseaseId").pivot("combined_max_phase").agg(F.countDistinct("approvedSymbol")).withColumnRenamed("null", "nullNelson")

# 3. Join both results to create the final DataFrame
result_df = all_sum_df.join(pivot_df_chembl, on="diseaseId", how="left")
result_df_final = result_df.join(pivot_df_nelson, on="diseaseId", how="left")

Merge with number of genetically associated genes for each disease

In [5]:
# Target-Disease indirect (propagated) evidence from Open Targets Platform (by data source)
evidence_sour_path = "gs://open-targets-data-releases/25.03/output/association_by_datasource_indirect/"
evidence_sour = spark.read.parquet(evidence_sour_path)

In [7]:
evidence_sour_ge_som = evidence_sour.filter(col('datatypeId').rlike("genetic_association|somatic_mutation"))

In [ ]:
# 1. Count unique targetId values per diseaseId (overall)
disease_targets = evidence_sour_ge_som.groupBy("diseaseId").agg(F.countDistinct("targetId").alias("diseaseTargets"))

# 2. Join with Pharmaprojects target disease counts
disease_targets = result_df_final.join(disease_targets, on="diseaseId", how="left")

Name for diseases' EFOs

In [9]:
disease_path = "gs://open-targets-data-releases/25.03/output/disease/"
disease = spark.read.parquet(disease_path)

In [10]:
def join_dataframes(initial_df: DataFrame, 
                    second_df: DataFrame, 
                    initial_key_column: str, 
                    second_key_column: str,
                    columns_to_join: list) -> DataFrame:
    """
    Joins two PySpark DataFrames on specified key columns.

    Args:
    initial_df (DataFrame): The initial PySpark DataFrame.
    second_df (DataFrame): The second PySpark DataFrame to join with.
    initial_key_column (str): The key column name in the initial DataFrame.
    second_key_column (str): The key column name in the second DataFrame.
    columns_to_join (list): List of column names from the second DataFrame to include in the join.

    Returns:
    DataFrame: The resulting DataFrame after the join.
    """

    # Selecting specified columns from the second DataFrame, including its key column
    second_df_selected = second_df.select([second_key_column] + columns_to_join)

    second_columns_to_join_with_alias = ["b." + col for col in columns_to_join]

    return initial_df.alias("a")\
        .join(second_df_selected.alias("b"), 
            on = initial_df[initial_key_column] == second_df_selected[second_key_column], 
            how='left')\
        .select("a.*", *second_columns_to_join_with_alias)\
        .persist()

In [11]:
disease_list = ["name", "therapeuticAreas"]

disease_targets_pharma_names = join_dataframes(disease_targets, disease, "diseaseId", "id", disease_list).persist()

25/06/04 13:16:39 WARN CacheManager: Asked to cache already cached data.


In [12]:
disease_targets_pharma_names.show()

+--------------+-------+----------+----+----+----+----+----+----------+--------+-------+--------+---------+-----------+--------------+--------------------+--------------------+
|     diseaseId|all_sum|nullChEMBL| 0.5| 1.0| 2.0| 3.0| 4.0|nullNelson|Launched|Phase I|Phase II|Phase III|Preclinical|diseaseTargets|                name|    therapeuticAreas|
+--------------+-------+----------+----+----+----+----+----+----------+--------+-------+--------+---------+-----------+--------------+--------------------+--------------------+
|   EFO_0004254|     10|      NULL|NULL|   1|   3|   2|   4|         8|    NULL|   NULL|    NULL|        2|       NULL|            13|membranous glomer...|       [EFO_0009690]|
|   EFO_1002027|      5|      NULL|NULL|NULL|   4|NULL|   1|         5|    NULL|   NULL|    NULL|     NULL|       NULL|          NULL|        osteomalacia|      [OTAR_0000006]|
|   EFO_0000569|     54|      NULL|NULL|  18|  27|NULL|   9|        54|    NULL|   NULL|    NULL|     NULL|       N

# Filtering based on amount os drug targets and associated genes

In [ ]:
# Step 1: Calculate sum of Drug Targets (DT) for each diseaseId
indications_list = disease_targets_pharma_names.withColumn(
    'sumDT',
    col('Launched') + col('Phase I') + col('Phase II') + col('Phase III') + col('Preclinical')
)

In [ ]:
# Step 2: Calculate rate of Drug Targets (DT) for each diseaseId
indications_list = indications_list.withColumn(
    'rateDT',
    (col('sumDT') / col('diseaseTargets')) * 100
)

In [22]:
# Step 3: Filter indications with rate of drug targets greater then 5% and number of genes greater then 500
indications_list_filt = indications_list.filter(
    (col('rateDT') >= 0.05) & (col('diseaseTargets') > 500)
)

In [23]:
indications_list_filt.count()

39

In [ ]:
indications_list_filt

In [24]:
# Step 4: Explode and count therapeutic areas
exploded = indications_list_filt.withColumn('therapeuticArea', explode(col('therapeuticAreas')))
value_counts_df = exploded.groupBy('therapeuticArea').count().withColumnRenamed('count', 'Count')
value_counts_df.show()

+---------------+-----+
|therapeuticArea|Count|
+---------------+-----+
|   OTAR_0000010|    4|
|   OTAR_0000018|    9|
|    EFO_0010285|    2|
|    EFO_0000540|   10|
|  MONDO_0045024|   22|
|    EFO_0005803|    6|
|   OTAR_0000017|    4|
|    EFO_0001379|    6|
|    EFO_0009690|    3|
|  MONDO_0024458|    1|
|   OTAR_0000006|    4|
|    EFO_0010282|    4|
|    EFO_0000618|    6|
|  MONDO_0002025|    3|
|    EFO_0009605|    2|
|    EFO_0000319|    3|
|   OTAR_0000020|    1|
+---------------+-----+



# Prepare table for supplementary

In [25]:
indications_list_filt_pandas = indications_list_filt.toPandas()
indications_list_filt_pandas.to_csv("outputs/indications_dt.csv", index=False)